In [3]:
import sys
import os
sys.path.insert(0, os.path.abspath('../src/'))

### Computing t-SNE

In [5]:
import cicids2017
import numpy as np
from pathlib import Path
import pandas as pd
from sklearn.manifold import TSNE
from umap import UMAP


RES_PATH = "BestGruLinearTL"
MODELPATH = Path(f"../res/{RES_PATH}/GruLinearTL.torch")
# DATASETPATH_CACHE = Path(f"../res/{RES_PATH}/cache")
DATASETPATH_CACHE = Path(f"../dataset/cache")
DATASETPATH = Path("../dataset/CICIDS2017_ntop.pkl")
TARGET_DSET = "DT"


netdata = []
df = cicids2017.load_dataset(DATASETPATH_CACHE)[TARGET_DSET]
days = np.unique(df.index.get_level_values("_time").day)
for d in days:
    daily_df = df[df.index.get_level_values("_time").day == d]
    netdata.append(daily_df)


net = cicids2017.configureAnchor(None, df, checkpoint=MODELPATH)
netdata = net.pointwise_embedding(netdata)
netdata = pd.concat(netdata)


# Remove truncated embeddings
netdata = netdata.loc[netdata["_embedding"].dropna().index]
tsne_input = np.stack(netdata["_embedding"].values)
netdata["_ebs2D"] = list(TSNE(n_components=2).fit_transform(tsne_input))

/Users/swedishhdcamera/Desktop/Projects/tesi_sabella/venv/lib/python3.8/site-packages/skorch/dataset.py:267: FutureWarning: Setting a random_state has no effect since cv is not a float. This will raise an error in a future. You should leave random_state to its default (None), or set cv to a float value.
  warnings.warn(


### DBScan

In [6]:
from sklearn.cluster import DBSCAN

cluster_algorithm = DBSCAN(eps=4, min_samples=5)

cluster_str = str(cluster_algorithm).split('(')[0]
clustering_input = np.stack(netdata["_ebs2D"].values)
clustering = cluster_algorithm.fit(clustering_input)
netdata["_clustering"] = clustering.labels_.astype(str)

### Anomaly detection

In [30]:
normal_cluster = netdata["_clustering"].mode().iloc[0]
netdata["_yhat"] = (netdata["_clustering"] == normal_cluster)
Y = netdata["_isanomaly"]
Y = (Y=="none").replace({True: 0, False: 1}).values
y_hat = netdata["_yhat"].replace({True: 0, False: 1}).values

In [31]:
from sklearn import metrics

# report = metrics.classification_report(y_attacks, y_hat)
metrics_rep = [ metrics.roc_auc_score,
                metrics.precision_score, metrics.recall_score,
                metrics.accuracy_score, metrics.f1_score]
for m in metrics_rep:
    mres = m(Y, y_hat)
    print(f"{m.__name__}(moday+attacks): {mres}")

tn, fp, fn, tp = metrics.confusion_matrix(Y, y_hat, normalize="all").ravel()
print("\n Confusion matrix")
print(f"\ttp: {tp} \tfp: {fp} \n\tfn: {fn} \ttn: {tn}")
# print(f"\n{report}")

roc_auc_score(moday+attacks): 0.8456554406703033
precision_score(moday+attacks): 0.8046934140802422
recall_score(moday+attacks): 0.7241144414168937
accuracy_score(moday+attacks): 0.9289617486338798
f1_score(moday+attacks): 0.7622803872355683

 Confusion matrix
	tp: 0.11389692489017465 	fp: 0.027643844423015106 
	fn: 0.04339440694310511 	tn: 0.8150648237437051
